# Learn OpenAI Whisper - Chapter 8
## Notebook 1: Cloning voices with tortoise-tts-fast

This notebook complements the book [Learn OpenAI Whisper](https://a.co/d/1p5k4Tg).

This notebook is based on the [TorToiSe-TTS-Fast](https://github.com/152334H/tortoise-tts-fast) project, which drastically boost the performance of [TorToiSe](https://github.com/neonbjb/tortoise-tts), without modifying the base models.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1zxZ7TCYr8hiU7ExY6QuXjPfVldWtcz-s)

## 1. Setting up the environment:
   - The code starts by cloning the "tortoise-tts-fast" repository and installing the required dependencies.
   - It uses `notebook_login()` from the `huggingface_hub` library to authenticate with Hugging Face.
   - The necessary imports are made, including `torch`, `torchaudio`, and modules from the `tortoise` package.

In [ ]:
%%capture loading_libraries
!git clone https://github.com/152334H/tortoise-tts-fast
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git
!pip install transformers==4.29.2
!pip install voicefixer==0.1.2

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

## 2. Initializing the TextToSpeech model:
   ```python
   from tortoise.api import TextToSpeech
   tts = TextToSpeech()
   ```
   - The `TextToSpeech` class is imported from the `tortoise.api` module.
   - An instance of `TextToSpeech` is created, which will download all the required models from the Hugging Face hub.


In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

In [ ]:
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.

#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
%cd tortoise-tts-fast
%ls tortoise/voices

## 3. Selecting a voice:
   - The code uses the `os` module to list all the available voice folders in the "tortoise/voices" directory.
   - It creates a dropdown widget using `Dropdown` from the `ipywidgets` library to allow the user to select a voice folder.
   - Another dropdown widget is created to select a specific voice file within the selected voice folder.
   - The selected voice can be played using `IPython.display.Audio`.

In [ ]:
import os
from ipywidgets import Dropdown

voices_dir = "/content/tortoise-tts-fast/tortoise/voices"

# Get a list of all directories in the voices directory
voice_names = os.listdir(voices_dir)

voice_folder = Dropdown(
    options=sorted(voice_names),
    description='Select a voice:',
    value='tom',
    disabled=False,
    style={'description_width': 'initial'},
)

voice_folder

In [ ]:
import os
from ipywidgets import Dropdown

voices_dir = f"/content/tortoise-tts-fast/tortoise/voices/{voice_folder.value}"

# Get a list of all directories in the voices directory
voice_files = os.listdir(voices_dir)

voice = Dropdown(
    options=sorted(voice_files),
    description='Select a voice:',
    # value='tom',
    disabled=False,
    style={'description_width': 'initial'},
)

voice

In [ ]:
#Pick one of the voices from the output above
IPython.display.Audio(filename=f'tortoise/voices/{voice_folder.value}/{voice.value}')

## 4. Generating speech with a selected voice:
   ```python
   text = "When the season changes, events and gatherings are in full bloom..."
   preset = "ultra_fast"
   voice = voice_folder.value
   voice_samples, conditioning_latents = load_voice(voice)
   gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
   torchaudio.save(generated_filename, gen.squeeze(0).cpu(), 24000)
   IPython.display.Audio(generated_filename)
   ```
   - The text to be spoken is defined in the `text` variable.
   - The `preset` variable determines the quality of the generated speech (options: "ultra_fast", "fast", "standard", "high_quality").
   - The selected voice is loaded using `load_voice` from `tortoise.utils.audio`, which returns `voice_samples` and `conditioning_latents`.
   - The `tts_with_preset` method of the `tts` object is called with the text, voice samples, conditioning latents, and preset to generate the speech.
   - The generated speech is saved as a WAV file using `torchaudio.save` and played using `IPython.display.Audio`.

In [ ]:
# This is the text that will be spoken.
text = "Words, once silent, now dance on digital breath, speaking volumes through the magic of text-to-speech." #@param {type:"string"}
# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "ultra_fast" #@param ["ultra_fast", "fast", "standard", "high_quality"]

In [ ]:
# Pick one of the voices from the output above
voice = voice_folder.value

#Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)

generated_filename = f'generated-{preset}-{voice}.wav'
torchaudio.save(generated_filename, gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(generated_filename)

## 5. Generating speech with a random voice:
   - Similar to the previous step, but with `voice_samples` and `conditioning_latents` set to `None`, which generates speech using a random voice.

In [ ]:
#@markdown Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
#@markdown (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('synthetized_voice_sample.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('synthetized_voice_sample.wav')

## 6. Using a custom voice:
   - The code allows the user to upload their own WAV files (6-10 seconds long) to create a custom voice.
   - It creates a custom voice folder using `os.makedirs` and saves the uploaded files in that folder.
   - The custom voice is then loaded and used to generate speech, similar to steps 4 and 5.

In [ ]:
#@markdown Optionally, upload use your own voice by running the next two cells. I recommend
#@markdown you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "custom"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

## 7. Combining voices:
   - The `load_voices` function is used to load multiple voices (in this case, 'pat' and 'william').
   - The `tts_with_preset` method is called with the combined voice samples and conditioning latents to generate speech with traits from both voices.

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['freeman', 'deniro'])

gen = tts.tts_with_preset("Words, once silent, now dance on digital breath, speaking volumes through the magic of text-to-speech.",
                          voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save('freeman_deniro.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('freeman_deniro.wav')